<a href="https://colab.research.google.com/github/fandeveloper4-ai/ProductivityProTech/blob/main/Discrete_Structures_Toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from collections import defaultdict, deque
import heapq
from typing import List, Tuple, Dict, Any, Generator, Set

# ==============================================================================
# 1. GRAPH THEORY: DIJKSTRA'S ALGORITHM
# Purpose: Find the shortest path from a starting node to all other nodes
# in a weighted, directed graph with non-negative edge weights.
# The graph is represented as an adjacency list: {node: {neighbor: weight}}
# ==============================================================================

def dijkstra(graph: Dict[Any, Dict[Any, float]], start_node: Any) -> Tuple[Dict[Any, float], Dict[Any, Any]]:
    """
    Computes the shortest path from a start node to all other nodes using Dijkstra's algorithm.

    Args:
        graph: A dictionary representing the graph: {node: {neighbor: weight}}.
        start_node: The starting node for pathfinding.

    Returns:
        A tuple containing:
        - distances: A dictionary of {node: shortest_distance}.
        - predecessors: A dictionary of {node: predecessor_node} to reconstruct the path.
    """
    # Priority queue stores (distance, node)
    priority_queue = [(0.0, start_node)]
    # Distances dictionary, initialized with infinity
    distances = {node: float('inf') for node in graph}
    distances[start_node] = 0.0
    predecessors = {node: None for node in graph}

    # Initialize nodes that might not have outgoing edges but are in the graph definition
    for node in graph.keys():
        if node not in distances:
            distances[node] = float('inf')
            predecessors[node] = None

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # If we found a longer path to current_node than already processed, skip
        if current_distance > distances[current_node]:
            continue

        # Iterate over neighbors
        for neighbor, weight in graph.get(current_node, {}).items():
            distance = current_distance + weight

            # Relaxation step
            if distance < distances.get(neighbor, float('inf')):
                distances[neighbor] = distance
                predecessors[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances, predecessors

# Helper to reconstruct the path
def reconstruct_path(predecessors: Dict[Any, Any], target_node: Any) -> List[Any]:
    """Reconstructs the shortest path from the predecessors dictionary."""
    path = deque()
    current = target_node
    while current is not None:
        path.appendleft(current)
        current = predecessors.get(current)

    # Check if the path starts at the source (i.e., path is not empty)
    if not path or path[0] == target_node and predecessors.get(target_node) is not None:
         # If target_node exists but has no predecessor, it might be the start node or unreachable.
        if len(path) == 1 and predecessors.get(target_node) is None:
             return list(path)
        return [] # Path is unreachable or ill-formed
    return list(path)


# ==============================================================================
# 2. COMBINATORICS: PERMUTATIONS, COMBINATIONS, AND FACTORIAL
# Purpose: Functions for fundamental counting principles.
# ==============================================================================

def factorial(n: int) -> int:
    """Calculates n! (n factorial) using a standard math library function."""
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    return math.factorial(n)

def combinations(n: int, k: int) -> int:
    """
    Calculates the number of combinations (n choose k): C(n, k).
    Formula: n! / (k! * (n-k)!)
    """
    if k < 0 or k > n:
        return 0
    if k == 0 or k == n:
        return 1
    if k > n // 2: # Optimization: C(n, k) = C(n, n-k)
        k = n - k

    # Use math.comb for robust calculation (Python 3.8+)
    if hasattr(math, 'comb'):
        return math.comb(n, k)

    # Fallback calculation (for compatibility/demonstration)
    res = 1
    for i in range(k):
        res = res * (n - i) // (i + 1)
    return res

def permutations(n: int, k: int) -> int:
    """
    Calculates the number of permutations: P(n, k).
    Formula: n! / (n-k)!
    """
    if k < 0 or k > n:
        return 0

    # Use math.perm for robust calculation (Python 3.8+)
    if hasattr(math, 'perm'):
        return math.perm(n, k)

    # Fallback calculation
    res = 1
    for i in range(k):
        res *= (n - i)
    return res

def generate_permutations(items: List[Any]) -> Generator[Tuple[Any, ...], None, None]:
    """Generates all permutations of a list of items using recursion."""
    # Base case: an empty list has one permutation (the empty tuple)
    if not items:
        yield ()
    else:
        # Recursive step: for each item, prepend it to all permutations of the remaining items
        for i, item in enumerate(items):
            remaining_items = items[:i] + items[i+1:]
            for p in generate_permutations(remaining_items):
                yield (item,) + p


# ==============================================================================
# 3. NUMBER THEORY: PRIMALITY, GCD, AND PRIME FACTORIZATION
# Purpose: Functions for integer properties and relationships.
# ==============================================================================

def is_prime(n: int) -> bool:
    """Checks if a positive integer n is a prime number."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def prime_factors(n: int) -> Dict[int, int]:
    """
    Computes the prime factorization of a positive integer n.
    Returns a dictionary of {prime_factor: exponent}.
    """
    factors = defaultdict(int)
    d = 2
    temp = n
    while d * d <= temp:
        while temp % d == 0:
            factors[d] += 1
            temp //= d
        d += 1
    if temp > 1:
        factors[temp] += 1
    return dict(factors)

def gcd(a: int, b: int) -> int:
    """
    Calculates the Greatest Common Divisor (GCD) of two integers using the Euclidean algorithm.
    """
    # Use math.gcd for standard implementation (Python 3.5+)
    if hasattr(math, 'gcd'):
        return math.gcd(a, b)

    # Fallback (Manual Euclidean Algorithm)
    while b:
        a, b = b, a % b
    return a

def lcm(a: int, b: int) -> int:
    """
    Calculates the Least Common Multiple (LCM) of two integers.
    Formula: LCM(a, b) = (|a * b|) / GCD(a, b)
    """
    if a == 0 or b == 0:
        return 0
    return abs(a * b) // gcd(a, b)


# ==============================================================================
# 4. SET THEORY: CARTESIAN PRODUCT
# Purpose: Computes the Cartesian product of multiple sets/lists.
# ==============================================================================

def cartesian_product(*sets: Set[Any]) -> Generator[Tuple[Any, ...], None, None]:
    """
    Computes the Cartesian product of a variable number of sets or lists.
    E.g., A x B x C.

    Args:
        *sets: Variable number of sets or lists.

    Yields:
        Tuples representing elements of the Cartesian product.
    """
    # Convert sets to lists to ensure consistent iteration order
    list_of_sets = [list(s) for s in sets]

    # Base case: one set (yields single-element tuples)
    if not list_of_sets:
        yield ()
        return

    # Use a recursive helper function
    def product_recursive(current_index: int, current_tuple: Tuple[Any, ...]):
        if current_index == len(list_of_sets):
            yield current_tuple
            return

        for item in list_of_sets[current_index]:
            yield from product_recursive(current_index + 1, current_tuple + (item,))

    yield from product_recursive(0, ())


# ==============================================================================
# EXAMPLE USAGE AND DEMONSTRATION
# ==============================================================================

def run_examples():
    print("==============================================")
    print("Discrete Structures Toolkit Demonstration")
    print("==============================================\n")

    # 1. Graph Theory Example (Dijkstra's)
    print("--- 1. Dijkstra's Algorithm (Shortest Path) ---")
    graph = {
        'A': {'B': 1.0, 'C': 4.0},
        'B': {'C': 2.0, 'D': 5.0, 'E': 1.0},
        'C': {'D': 1.0},
        'D': {'E': 3.0},
        'E': {}
    }
    start = 'A'
    distances, predecessors = dijkstra(graph, start)
    target = 'E'
    path = reconstruct_path(predecessors, target)

    print(f"Graph: {graph}")
    print(f"Shortest distance from {start} to all nodes: {distances}")
    print(f"Shortest path from {start} to {target}: {path} (Cost: {distances.get(target)})")
    print("-" * 40)

    # 2. Combinatorics Examples
    print("--- 2. Combinatorics ---")
    n_items = 5
    k_selected = 3
    print(f"Factorial of {n_items} (5!): {factorial(n_items)}")
    print(f"Combinations C({n_items}, {k_selected}) (5 choose 3): {combinations(n_items, k_selected)}")
    print(f"Permutations P({n_items}, {k_selected}): {permutations(n_items, k_selected)}")

    items_to_permute = ['X', 'Y', 'Z']
    perms = list(generate_permutations(items_to_permute))
    print(f"All permutations of {items_to_permute} ({len(perms)} total): {perms}")
    print("-" * 40)

    # 3. Number Theory Examples
    print("--- 3. Number Theory ---")
    num1, num2 = 120, 42
    print(f"GCD of {num1} and {num2}: {gcd(num1, num2)}")
    print(f"LCM of {num1} and {num2}: {lcm(num1, num2)}")

    num_to_factor = 360
    print(f"Is 17 prime? {is_prime(17)}")
    print(f"Is 27 prime? {is_prime(27)}")
    print(f"Prime factorization of {num_to_factor} (360): {prime_factors(num_to_factor)}")
    print("-" * 40)

    # 4. Set Theory Example (Cartesian Product)
    print("--- 4. Set Theory (Cartesian Product) ---")
    set_A = {'Red', 'Blue'}
    set_B = {1, 2, 3}
    set_C = {'S', 'M'}
    prod_elements = list(cartesian_product(set_A, set_B, set_C))
    print(f"Set A: {set_A}")
    print(f"Set B: {set_B}")
    print(f"Set C: {set_C}")
    print(f"Cartesian Product (A x B x C) has {len(prod_elements)} elements:")
    print(prod_elements)
    print("==============================================")


if __name__ == '__main__':
    run_examples()